In [2]:
import cv2
import numpy as np
from skimage.filters import threshold_local

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    return edged

def find_document_contour(image):
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approx) == 4:
            return approx
    return None

def transform_perspective(image, contour):
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def enhance_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    T = threshold_local(gray, 11, offset=10, method="gaussian")
    enhanced = (gray > T).astype("uint8") * 255
    return enhanced

def scan_image(image):
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    image = cv2.resize(image, (int(image.shape[1] / ratio), 500))
    edged = preprocess_image(image)
    contour = find_document_contour(edged)
    if contour is None:
        print("Could not find document contour.")
        return None
    warped = transform_perspective(orig, contour * ratio)
    enhanced = enhance_image(warped)
    return enhanced



In [3]:
import sys
import os
import cv2
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QFileDialog, QVBoxLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import numpy as np
from skimage.filters import threshold_local

class ScannerApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        self.layout = QVBoxLayout()
        
        self.loadButton = QPushButton('Load Image')
        self.loadButton.clicked.connect(self.loadImage)
        self.layout.addWidget(self.loadButton)
        
        self.scanButton = QPushButton('Scan Document')
        self.scanButton.clicked.connect(self.scanDocument)
        self.layout.addWidget(self.scanButton)
        
        self.imageLabel = QLabel()
        self.layout.addWidget(self.imageLabel)
        
        self.setLayout(self.layout)
        self.setWindowTitle('Document Scanner')
        self.setGeometry(100, 100, 800, 600)
    
    def loadImage(self):
        options = QFileDialog.Options()
        fileName, _ = QFileDialog.getOpenFileName(self, 'Open Image File', '', 'Images (*.png *.jpg *.jpeg *.bmp)', options=options)
        if fileName:
            self.image = cv2.imread(fileName)
            self.displayImage(self.image)
    
    def scanDocument(self):
        if hasattr(self, 'image'):
            scanned_image = scan_image(self.image)
            if scanned_image is not None:
                self.displayImage(scanned_image, scanned=True)
                self.saveImage(scanned_image)
    
    def displayImage(self, img, scanned=False):
        if scanned:
            height, width = img.shape
            qImg = QImage(img.data, width, height, width, QImage.Format_Grayscale8)
        else:
            height, width, channel = img.shape
            bytesPerLine = 3 * width
            qImg = QImage(img.data, width, height, bytesPerLine, QImage.Format_RGB888)
        
        pixmap = QPixmap.fromImage(qImg)
        self.imageLabel.setPixmap(pixmap.scaled(self.imageLabel.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
    
    def saveImage(self, img):
        desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')
        output_path = os.path.join(desktop_path, 'scanned_document.png')
        cv2.imwrite(output_path, img)
        print(f"Scanned document saved at: {output_path}")

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
    return edged

def find_document_contour(image):
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approx) == 4:
            return approx
    return None

def transform_perspective(image, contour):
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def enhance_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    T = threshold_local(gray, 11, offset=10, method="gaussian")
    enhanced = (gray > T).astype("uint8") * 255
    return enhanced

def scan_image(image):
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    image = cv2.resize(image, (int(image.shape[1] / ratio), 500))
    edged = preprocess_image(image)
    contour = find_document_contour(edged)
    if contour is None:
        print("Could not find document contour.")
        return None
    warped = transform_perspective(orig, contour * ratio)
    enhanced = enhance_image(warped)
    return enhanced

if __name__ == '__main__':
    app = QApplication(sys.argv)
    scanner = ScannerApp()
    scanner.show()
    sys.exit(app.exec_())


Scanned document saved at: C:\Users\HOSSEIN\Desktop\scanned_document.png


SystemExit: 0

C:\Users\HOSSEIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
